CrewAI - Gemini 01
https://youtu.be/B8s-FyN4UeE?si=JF6BfX1aOa8tdxl3

CrewAI 是一個自治代理框架，使您能夠建立一組代理來完成複雜的任務。 它在功能上與 AutoGen 非常相似。 您還將學習如何將 Google 的免費 G​​emini API 與 CrewAI 結合使用。

影片是用 Google Colab 來完成的。所以，同時用本地的 jupyter-lab 跟 Google Colab 同時來操作看看囉

In [ ]:
# 安裝crewai, duckduckgo-search, langchain-google-genai
!pip install crewai duckduckgo-search langchain-google-genai

In [1]:
import os
from crewai import Agent, Task, Crew, Process
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.llms import OpenAI, Ollama
from langchain_openai import ChatOpenAI

設定 LLM

In [2]:
# 設定4個 LLM
# crewai 使用 Ollama 的方法：windows 只需要打開 ubuntu 視窗即可，不需要執行 litellm；Mac 則甚至不需要啟動任何程式，只要有安裝好 Ollama 即可調用。
# 與 autogen 不同，autogen 要調用 Ollama 需要執行 litellm，多一道步驟，比較麻煩
gpt35 = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7),
gpt4 = ChatOpenAI(model_name="gpt-4", temperature=0.7),
# Ollama 大模型，可以看要用哪一個
ollama = Ollama(model="openhermes_assistant")  # 這是用 modelfile 建立的
ollama_openhermes = Ollama(model="openhermes")

gemini = ChatGoogleGenerativeAI(model="gemini-pro",
                             verbose = True,
                             temperature = 0.6,
                             google_api_key = os.environ["GOOGLE_API_KEY"]
                             )


設定 tools

In [3]:
# 設定工具 tools
# Make sure to Install duckduckgo-search for this example:
# !pip install -U duckduckgo-search
# 最簡易型的工具定義方式
from langchain.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()

# 從 tools 模組中引入需要的工具
from tools.browser_tools import BrowserTools
from tools.calculator_tools import CalculatorTools
from tools.file_tools import FileTools
from tools.search_tools import SearchTools
from tools.sec_tools import SECTools
# 下面添加 tools 時，必須用 類別.方法 加入工具
# BrowserTools.scrape_and_summarize_website
# CalculatorTools.calculate
# FileTools.write_file
# SearchTools.search_internet、SearchTools.search_news
# SECTools.search_10q、SECTools.search_10k
# 下面的是依樣畫葫蘆自行加入2行定義類別，親測可以匯入
from tools.find_papers_arxiv import FindPapersArxiv
from tools.get_top_hackernews_stories import GetTopHackernewsStories
# 然後就可以用下面的函式來使用工具，已測試可以~~~
# FindPapersArxiv.search_arxiv
# GetTopHackernewsStories.get_top_hackernews_stories

接下來開始定義 Agent，這部分取決於你想要用多少個 Agent，每個 Agent 都有不同的 role(角色)，不同的 goal(目標)及不同的 backgstory(背景故事)，或者不同的 system prompt(系統提示詞)。

In [4]:
researcher = Agent(
    role = 'Senior Research Analyst',
    goal = 'Uncover cutting-edge development in AI and data science',
    backstory = """ You work in a leading tech think tank.
    Your expertise lies in identifying emerging trends.
    You have a knack for dissecting complex data and presenting actionable insights.""",
    verbose = True,
    allow_delegation = False,
    tools=[
            search_tool, 
            BrowserTools.scrape_and_summarize_website, 
            CalculatorTools.calculate, 
            FileTools.write_file,
            SearchTools.search_internet, SearchTools.search_news,
            SECTools.search_10q, SECTools.search_10k,
            FindPapersArxiv.search_arxiv,
            GetTopHackernewsStories.get_top_hackernews_stories
        ]
)
writer = Agent(
    role='Tech Content Strategist',
    goal='Craft compelling content on tech advancements',
    backstory="""You are a renowned Content Strategist, known for
    your insightful and engaging articles.
    You transform complex concepts into compelling narratives.""",
    verbose=True,
    allow_delegation=True,
    tools=[
            search_tool, 
            BrowserTools.scrape_and_summarize_website, 
            CalculatorTools.calculate, 
            FileTools.write_file,
            SearchTools.search_internet, SearchTools.search_news,
            SECTools.search_10q, SECTools.search_10k,
            FindPapersArxiv.search_arxiv,
            GetTopHackernewsStories.get_top_hackernews_stories
        ]
)

In [5]:
# 上面可以先不指定用哪一個 llm，這裡再指定就可以
researcher.llm = gemini
writer.llm = ollama

接下來提供這些 Agent 要執行的 task(任務列表) 

In [18]:
task1 = Task(
  description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.
  Your final answer MUST be a full analysis report""",
  agent=researcher
)

task2 = Task(
  description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant AI advancements.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, avoid complex words so it doesn't sound like AI.
  Your final answer MUST be the full blog post of at least 5 paragraphs.
  """,
  agent=writer
)

task3 = Task(
  description="""express the final blog post provided in traditional chinese。 and write it to a file AI_2024.md""",
  agent=writer
)

使用順序過程實例化您的 Agent 成員(crew)

In [19]:
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2, task3],
  verbose=2, # You can set it to 1 or 2 to different logging levels
)

Overriding of current TracerProvider is not allowed


讓你的 Agent成員(crew) 開始工作！並輸出成果！

In [20]:
result = crew.kickoff()

print("######################")
print(result)

[DEBUG]: Working Agent: Senior Research Analyst
[INFO]: Starting Task: Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.
  Your final answer MUST be a full analysis report


> Entering new CrewAgentExecutor chain...


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


Thought: Do I need to use a tool? Yes
Action: Search the internet
Action Input: latest advancements in AI in 2024Tool search the internet has been used.
 

Title: 10 top AI and machine learning trends for 2024 | TechTarget
Link: https://www.techtarget.com/searchenterpriseai/tip/9-top-AI-and-machine-learning-trends
Snippet: 10 top AI and machine learning trends for 2024 · 1. Multimodal AI · 2. Agentic AI · 3. Open source AI · 4. Retrieval-augmented generation · 5.

-----------------
Title: What's next for AI in 2024 - MIT Technology Review
Link: https://www.technologyreview.com/2024/01/04/1086046/whats-next-for-ai-in-2024/
Snippet: In 2024, generative AI might actually become useful for the regular, non-tech person, and we are going to see more people tinkering with a ...

-----------------
Title: How 2024 Will Be A.I.'s 'Leap Forward' - The New York Times
Link: https://www.nytimes.com/2024/01/08/technology/ai-robots-chatbots-2024.html
Snippet: A.I. is set to advance at a rapid rate, be